In [6]:
from qiskit import *
import numpy as np
import math

### Problema práctico 1 - Puertas clásicas con puertas cuánticas

Hemos visto durante este inicio del curso que las puertas lógicas cuánticas generalizan a las puertas clásicas, lo que quiere decir que con un circuito cuántico podemos representar cualquier circuito clásica. Para demostrar esto, crear, a partir de un circuito cuántico, las puertas $XOR$ y $NAND$. Aquí teneis un ejemplo para la puerta $NOT$:

In [ ]:
def NOT(inp):

    qc = QuantumCircuit(1, 1) # this is where the quantum program goes
    
    # Codificaremos 0 -> |0> y 1 -> |1>. Por tanto, si nos entra un cero, aplicamos una puerta X 
    # para girar el qubit inicial.
    if inp==1:
        qc.x(0)
        
    # En este caso, el problema es sencillo, ya que la puerto NOT es equivalente a la puerta cuantica x:
    qc.x(0)
    
    # Medimos el estado del qubit
    qc.measure(0, 0)
    
    # Y corremos el circuito con el simulador
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc,backend)
    # Sacamos los valores de la ejecución del circuito
    output = next(iter(job.result().get_counts()))
    
    return output

print([NOT(0), NOT(1)])

*Pista:* Para la $XOR$, solo necesitamos una puerta cuántica. Para la $NOR$, con hacer dos medidas nos basta!

In [1]:
# Solución

### Problema práctico 2 - Teleportación cuántica

Uno de los algoritmos más sencillos pero con más utilidad es el de teleportación cuántica. Es la base de lo que llamamos ahora comunicación cuántica. Permite enviar un estado, digamos $|\psi\rangle = \alpha|0\rangle+\beta|1\rangle$ de un punto A a un punto B. Existe un teorema en la física cuántica llamado teorema de no clonación, por el cual es imposible generar una copia exacta de un estado cuántico sin destruirlo. Esto tiene que ver con la decoherencia, ya que si quiero medir el estado $|\psi\rangle$ para poder generar una copia, dicha medición destruye directamente el estado. Para obetener los valores exactos de $\alpha$ y $\beta$, necesitaría medir el estado miles de veces para que las probabilidades de medir cada uno de los estado $|0\rangle$ y $|1\rangle$ convergiera a $\alpha$ y $\beta$.

Existe una manera de solucionar este problema, y es mediante el uso del entrelazamiento. Vamos a explicar el funcionamiento teórico del algoritmo y vuestra tarea sera la de recrear el algoritmo usando Qiskit.

**El algoritmo:**
El objetivo es mandar un qbit en el estado $|\psi\rangle$ desde el laboratorio de Alice hasta el de Bob
- **Paso 1:** Alice y Bob creando un par de qbit entrelazados, $q_A$ y $q_B$ y cada uno se guarda uno de ellos.
- **Paso 2:** Alice aplica una puerta CNOT al qbit $q_A$, controlada por $|\psi\rangle$.
- **Paso 3:** Alice aplica una puerta Hadamard a $|\psi\rangle$ y mide los dos qubits $q_A$ y $|\psi\rangle$.
- **Paso 4:** Alice llama a Bob y le comunica el resultado de su medición. Dependiendo del resultado de la medición, Bob aplica una puerta y otra, de la manera siguiente:

| $q_A$ | $|\psi\rangle$ | Puerta de Bob |
|----|----|---------|
| $0$ | $0$ | No hacer nada |
| $0$ | $1$ | Aplicar la puerta $X$ |
| $1$ | $0$ | Aplicar la puerta $Z$ |
| $1$ | $1$ | Aplicar las puertas $ZX$|

Y ya está! Con esto Bob ha transformado su qbit $q_B$ al estado $|\psi\rangle$!

En nuestro caso, el qubit a teletransportar va a ser el qubit 0 del siguiente circuito, el qubit 1 será $q_A$ y el qubit 2 será $q_B$.

In [45]:
def secret_unitary(circuit, qubit):
    for _ in range(4):
        coin = np.random.rand()
        if coin < 0.3:
            circuit.h(qubit)
        elif coin > 0.3 and coin < 0.6:
            circuit.y(qubit)
        else: 
            circuit.x(qubit)
            
q = QuantumRegister(4) # Vamos a considerar un qubit extra, a parte de los 
                       # tres necesarios. Veremos mas tarde su utilidad.
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)

secret_unitary(qc, q[0])
qc.barrier()

Ahora es vuestro turno! Crear el circuito en la siguiente celda. En nuestro caso, seguimos teniendo acceso al estado $|\psi\rangle$ y $q_A$, por lo que podemos sustituir las llamadas telefónicas por puertas controladas, teniendo en cuenta la tabla de más arriba.

In [2]:
# Solución

Una vez habeis creado el circuito, vamos a chequear que el estado actual de $q_B$ es exactamente el de $|\psi\rangle$. Existe multiples manera de hacerlo, pero una que es especilamente interesante cuando estamos trabajando con qubits es hacer un [*swap test*](https://en.wikipedia.org/wiki/Swap_test). El swap test actua de la manera siguiente: consideramos dos estados iniciales $|\psi\rangle$ y $|\phi\rangle$. Consideramos además un qubit auxiliar inicializado en $|0\rangle$. Si $|\langle \psi|\phi\rangle|^2=1$, es decir son el mismo estado, entonces la probabilidad de medir 0 en el ancilla qubit es 1. Si por el contrario no son iguales, la probabilidad de medir 0 es 1/2. Podemos encontrar esta puerta en qiskit como `cswap(ctl, tgt1, tgt2)`.

Utilizad como control el qubit que nos hemos utilizado hasta ahora y coger como target qubits $|\phi\rangle$ y $q_B$. Luego, medid el valor de este último qubit para ver si el algoritmo de teleportación ha funcionado.

In [3]:
# Solución

### Problema práctico 3 - Transformada de Fourier cuántica

En este problema trataremos de implementar las transformada de Fourier cuántica que hemos estudiado de manera teórica. Para ello, primero tenemos que definir la puerta $CROT$. Como ya hemos visto, se trata de una puerta que rota un qubit dependiendo de el valor del qubit de control. En `qiskit` encontramos la puerta de rotación $CU_1$, definida como
$$
CU_1(\theta) = 
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{i\theta} \\
\end{pmatrix}
$$

En el caso de la puerta $CROT_k$, el ángulo de rotación es $\theta = 2\pi/2^k = \pi/2^{k−1}$. Un ejemplo de implementación de $CROT_2$ sería:

`quantum_circuit.cu(math.pi/2.0, quantum_register[0], quantum_register[1])`

Vuestra tare consiste en lo siguiente:
- Crear un circuito de 3 qubits.
- Aplicar la función input_state al circuito. Esta función crea un estado para el cual la QFT da como estado resultando el estado $1$ (en el caso de tres qubits, $|001\rangle$).
- Implementar la QFT a partir del circuito que habeís escrito.
- Utilizando el `qasm_simulator`, medir el resultado del circuito:

In [1]:
def input_state(circ, q, n):
    """n-qubit input state for QFT that produces output 1."""
    for j in range(n):
        circ.h(q[j])
        circ.u1(-math.pi/float(2**(j)), q[j])

In [4]:
# Solución

### Problema práctico 4 - Estimación de fase cuántica

El objetivo de este problema es extraer el valor $\theta$ dado una operación unitaria $U$ y un estado $|\psi\rangle$ de manera que $U|\psi\rangle=e^{2\pi i\theta}|\psi\rangle$. Para ello debeís utilizar el algoritmo de estimación de fase. En este ejercicio vamos a considerar $U=Z$ y $|\psi\rangle=|1\rangle$. Vamos a solucionar el problema con  $n=2$ qubits ancillas.
- Paso 1: dibujar el circuito, identificando las diferentes puertas a implementar y cual es el output esperado del circuito.
- Paso 2: calcular la forma de cada una de las puertas controladas a calcular.
- Paso 3: Implementar el circuito y apartir de este sacar el valor de $\theta$. Utilizar tanto el `qasm_simulator` como los chips cuánticos.
    - Primero, definir el circuito con 2 ancillas y 1 qubit extra, sobre el cual tendreís que aplicar una puerta $X$ para transformarlo al estado $|1\rangle$. Definir el circuito de tal manera que el qubit $|\psi\rangle$ este en la última posición.
    - Aplicar el algoritmo. Para la QFT inversa, podeís utilizar la siguiente función `qft_inversa` que encontrareís más abajo.
    - Medir el estado de las ancillas, pasar el valor del qubit de binario a decimal y sacar el valor de $\theta$.

In [13]:
def qft_inversa(circuito, q, n):
    '''Inputs:
        - circuito: QuantumCircuit
        - q: QuantumRegister
        - n: Numero de ancillas '''
    for j in range(n):
        k = (n-1) - j
        for m in range(k):
            circuito.cu1(-math.pi/float(2**(k-m)), q[k], q[m])
        circuito.h(q[k])

In [5]:
# Solución